# Wordle Solver

In [1]:
# import functions
from wordle_helpers import *

In [2]:
# help(main)

In [3]:
# Main function that returns a list of possible words
green_letters = 'EOU'
green_positions = [5,2,3,]
yellow_letters = 'RS'
yellow_positions = [2,1,]
bad_letters = 'CANGH'

# Main function that returns a list of possible words
words = main(green_letters, green_positions, yellow_letters, yellow_positions, bad_letters)
print(words)

['rouse']


In [4]:
# Repetitive letters
letters_df = repetitive_letters(wordle_list=words)
print(letters_df.to_string(index=False))

Letters  Count
      R      1
      O      1
      U      1
      S      1
      E      1


#### Choosing next word

In [5]:
# Main function that returns a list of possible words
green_letters = ''
green_positions = []
yellow_letters = 'SOGUH'
yellow_positions = []
bad_letters = ''

# Main function that returns a list of possible words
words = main(
    green_letters,
    green_positions,
    yellow_letters,
    yellow_positions,
    bad_letters,
    wordle_list_path="wordle-possible-words",
)
print(words)

['sough']


In [6]:
# help(score)

In [7]:
df = load_data(file_name='wordle_ranking.csv')
df

,Names,Games_Won
0,Murilo,168
1,Barbara,74
2,Draw,170


In [8]:
# Update score
score_df = score()
score_df

,Names,Games_Won
0,Murilo,169
1,Barbara,74
2,Draw,170


In [9]:
# Print results
print(score_df.to_string(index=False))

  Names  Games_Won
 Murilo        169
Barbara         74
   Draw        170


In [10]:
# help(reset_score)

#### Set custom score

In [11]:
# help(set_score)

In [12]:
# # Set custom score
# df = set_score(m_score=168, b_score=74, draw_score=168)
# df

#### Reset score

In [13]:
## Reset score
# reset_score()